In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os

In [3]:
# Function to load and preprocess images from a directory
def load_data(data_dir, img_size):
    images = []
    labels = []
    for category in os.listdir(data_dir):
        label = 1 if category == "Stroke" else 0  # Label 1 for stroke and 0 for no stroke
        category_path = os.path.join(data_dir, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path)  # Read image
            img = cv2.resize(img, (img_size, img_size))  # Resize to the desired size
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

In [5]:
# Load data
data_dir = "./Brain_Data_Organised"  # Directory containing 'stroke' and 'normal' folders
img_size = 224  # Image size (224x224 for standard CNN input)
images, labels = load_data(data_dir, img_size)

In [7]:
labels

array([0, 0, 0, ..., 1, 1, 1])

In [9]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [11]:
# Validation data generator (just rescale)
datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
# Prepare generators
train_generator = datagen.flow(X_train, y_train, batch_size=32)
val_generator = datagen.flow(X_val, y_val, batch_size=32)

In [15]:
# Create a CNN model
model = Sequential()

In [17]:
# 1st Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu', input_shape=(img_size, img_size, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3),padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(128, kernel_size=(3, 3),padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

C:\Users\shubh\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Flatten the feature maps before passing to Dense layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))  # Dropout to reduce overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))

# Output layer (binary classification: stroke or no stroke)
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,178,177 (42.64 MB)

 Trainable params: 11,177,729 (42.64 MB)

 Non-trainable params: 448 (1.75 KB)

In [23]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

C:\Users\shubh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 188s 3s/step - accuracy: 0.5563 - loss: 6.8910 - val_accuracy: 0.5150 - val_loss: 2.0825
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 174s 3s/step - accuracy: 0.6810 - loss: 1.0275 - val_accuracy: 0.6028 - val_loss: 3.0164
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 179s 3s/step - accuracy: 0.7691 - loss: 0.5662 - val_accuracy: 0.6028 - val_loss: 7.8978
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 175s 3s/step - accuracy: 0.7936 - loss: 0.4448 - val_accuracy: 0.6168 - val_loss: 2.8385
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 173s 3s/step - accuracy: 0.8855 - loss: 0.2581 - val_accuracy: 0.5309 - val_loss: 0.9290
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 167s 3s/step - accuracy: 0.8783 - loss: 0.2746 - val_accuracy: 0.4451 - val_loss: 1.4197
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.9110 - loss: 0.1886 - val_accuracy: 0.4671 - val_loss: 1.2874
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 169s 3s/step - accuracy: 0.9299 - loss: 0.1557 - val_accuracy: 0.7745 - v

In [33]:
# Save the model for later use in the web app
model.save('brain_stroke_model.keras')

In [27]:
# Evaluate the model
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 374ms/step - accuracy: 0.8274 - loss: 0.4080
Validation Accuracy: 81.44%


In [29]:
# Import necessary libraries
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np

# Predict on the validation set
y_pred_prob = model.predict(val_generator)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary (0 or 1)

# Get the true labels
y_true = y_val[:len(y_pred)]  # Make sure the length of y_val matches predictions

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate classification report (precision, recall, F1-score)
class_report = classification_report(y_true, y_pred, target_names=["No Stroke", "Stroke"])
print("Classification Report:")
print(class_report)

# Calculate ROC-AUC score
roc_auc = roc_auc_score(y_true, y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.4f}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 374ms/step
Confusion Matrix:
[[157 148]
 [ 85 111]]
Classification Report:
              precision    recall  f1-score   support

   No Stroke       0.65      0.51      0.57       305
      Stroke       0.43      0.57      0.49       196

    accuracy                           0.53       501
   macro avg       0.54      0.54      0.53       501
weighted avg       0.56      0.53      0.54       501

ROC-AUC Score: 0.5457
